In [2]:
%run preprocess_data.ipynb

done with change concat
length df_tweets_all: 101017
0    69738
1    31279
Name: label_all, dtype: int64
done
replaced = hello world this replaced_url is a sentence replaced_url with replaced_url urls replaced_url
done


In [ ]:
import ktrain
from ktrain import text
from keras.models import load_model

In [21]:
outpath_model = '../models'
outpath_predictor = '../predictor'

filename = "df_tweets_all"+(time.strftime("%Y%m%d")+".csv")

predictorname  = "dectet_hatespeech_full_ep_3_"+(time.strftime("%Y%m%d"))

print(outpath_predictor + "/" + predictorname)

../predictor/dectet_hatespeech_full_ep_3_20200303


In [12]:
%time
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(df_tweets_all,
                                                                  'tweet_tidy_1',
                                                                   maxlen=125, 
                                                                   preprocess_mode='bert',
                                                                   label_columns =['label_all'])

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs
preprocessing train...
language: en


preprocessing test...
language: en


In [13]:
model = text.text_classifier('bert', 
                             (x_train, y_train), 
                             preproc=preproc)

Is Multi-Label? False
maxlen is 125
done.


In [14]:
learner = ktrain.get_learner(model,train_data=(x_train, y_train), 
                             val_data=(x_test, y_test), 
                             batch_size=64)

In [60]:
learner.lr_find()

simulating training for different learning rates... this may take a few moments...
Epoch 1/1024
 6720/90915 [=>............................] - ETA: 9:25:02 - loss: 0.0564 - acc: 0.9812

In [16]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Train on 90915 samples, validate on 10102 samples
Epoch 1/3
90915/90915 [==============================] - 38065s 419ms/step - loss: 0.2633 - acc: 0.8889 - val_loss: 0.1880 - val_acc: 0.9241
Epoch 2/3
90915/90915 [==============================] - 38902s 428ms/step - loss: 0.1651 - acc: 0.9353 - val_loss: 0.1708 - val_acc: 0.9337
Epoch 3/3
90915/90915 [==============================] - 43543s 479ms/step - loss: 0.0903 - acc: 0.9673 - val_loss: 0.1533 - val_acc: 0.9479


In [17]:
learner.validate(val_data=(x_test, y_test), 
               #  class_names=['label_all'],
                print_report=True)

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      6988
           1       0.93      0.90      0.91      3114

    accuracy                           0.95     10102
   macro avg       0.94      0.94      0.94     10102
weighted avg       0.95      0.95      0.95     10102



array([[6768,  220],
       [ 306, 2808]])

In [18]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [19]:
data = [ 'you stupid son of a bitch',
         'Tyou can suck my dick your black nigger',
        'fuck off',
        'Nice to see you jewish bastrad',
        'Black mother fucker',
        '@USER Someone should veTaken" this piece of shit to a volcano. 😂"',
        '@USER She should ask a few native Americans what their take on this is.',
        '@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL'
         ]

In [22]:
predictorname

'dectet_hatespeech_full_ep_3_20200303'

In [23]:
model.save(predictorname)

In [61]:
predictor.save(predictorname)

In [24]:
predictor.predict_proba(data)

array([[0.00101226, 0.99898773],
       [0.00113858, 0.99886143],
       [0.14234313, 0.85765684],
       [0.98801243, 0.01198756],
       [0.01170991, 0.9882901 ],
       [0.9227954 , 0.07720462],
       [0.9808138 , 0.01918627],
       [0.20315288, 0.7968471 ]], dtype=float32)